# Parse Projects `xlsx`

In [1]:
import json

import dotenv
import polars as pl
from utils import get_ids

dotenv.load_dotenv("../../.env")

True

In [2]:
df = pl.read_excel("../data/phase2/projects_clean.xlsx")
# parse money
# df = df.with_columns(
#     pl.col("Project Value").str.strip_prefix("$").str.replace_all(",", "").cast(pl.Float32)
# )
df.head()

​Project Name,​Project Description,​Project Status,Objective,Project Value,Project Needs,Operates in,Source Country,SDGs,​CCSA Pillar,Organization Type,More Info
str,str,str,str,f64,str,str,str,str,str,str,str
"""Drinking Water…","""Water distribu…","""In Execution""","""This project h…",120000.0,"""Grant Catalyti…","""Jamaica""","""The United Sta…","""SDG 12 - Respo…","""1.5% New Green…","""For-profit""",null
"""Phase 1: Solar…","""Caribbean nati…","""Completed""","""This project h…",50000.0,"""Grant Catalyti…","""Regional; Trin…","""Trinidad & Tob…","""SDG 7 - Afford…","""90% Renewable …","""For-profit""",null
"""Electric Urban…","""This project p…","""Start-up to Ea…","""Project is bei…",555000.0,"""Equity""","""Belize""","""Belize""","""SDG 7 - Afford…","""90% Renewable …","""For-profit""",null
"""Phase 2: Build…","""A regional ble…","""Start-up to Ea…","""Incubation""",2.18e8,"""Other""","""Anguilla; Anti…","""Barbados""","""SDG 7 - Afford…","""90% Renewable …","""For-profit""",null
"""Regenerating t…","""The objective …","""In Execution""","""This project h…",50000.0,"""Grant Pre-Inve…","""Antigua and Ba…","""Europe""","""SDG 13 - Clima…","""30 x 30 Nature…","""Nonprofit""",null


In [11]:
df = df.rename(lambda x: x.strip().replace("\u200b", ""))

In [20]:
df.select(
    pl.col("Project Status").unique(), pl.col("Project Status").unique_counts().alias("counts")
)

Project Status,counts
str,u32
"""In Execution""",11
"""Start-up to Ea…",3
"""Completed""",10
"""Pre-operationa…",89
"""Scaling or Exp…",5
"""Mature Project…",3


## SDGs

In [21]:
sdgs = df.select(pl.col("SDGs").str.split("; ").list.explode()).unique().to_numpy().ravel()
sdgs = sorted(sdgs, key=lambda x: int(x[4:6]))
print(json.dumps([{"name": e} for e in sdgs]))

[{"name": "SDG 7 - Affordable and clean energy"}, {"name": "SDG 8 - Decent work and economic growth"}, {"name": "SDG 9 - Industry Innovation and Infrastructure"}, {"name": "SDG 11 - Sustainable Cities and Communities"}, {"name": "SDG 12 - Responsible production and consumption"}, {"name": "SDG 13 - Climate Action"}, {"name": "SDG 14 - Life below water"}, {"name": "SDG 15 - Life on land"}, {"name": "SDG 17 - Partnership for the goals"}]


## Pillars

In [23]:
pilars = df.select(pl.col("CCSA Pillar")).unique()
print(json.dumps([{"name": e} for e in list(pilars.to_numpy().ravel())]))

[{"name": "1.5% New Green Jobs for Physical & Economic Resilience"}, {"name": "30 x 30 Nature Based Solutions"}, {"name": "Climate Smart Map"}, {"name": "90% Renewable Energy for All"}]


## Countries

In [27]:
countries = list(
    df.select(pl.col("Operates in").str.split("; ").list.explode()).unique().to_numpy().ravel()
)
countries

['Bahamas',
 'Dominica',
 'Suriname',
 'US Virgin Islands',
 'Antigua and Barbuda',
 'Montserrat',
 'Puerto Rico',
 'Grenada',
 'Guyana',
 'Cayman',
 'St. Kitts & Nevis',
 'Trinidad & Tobago',
 'British Virgin Islands',
 'Bermuda',
 'Barbados',
 'Curacao',
 'Panama',
 'Bonaire',
 'Honduras',
 'Turks & Caicos Islands',
 'Haiti',
 'Jamaica',
 'Regional',
 'Japan',
 'St. Vincent & the Grenadines',
 'Dominican Republic',
 'Aruba',
 'Belize',
 'Anguilla',
 'Saint Lucia',
 'Costa Rica',
 'Mexico']

## get `IDs`

Previous SDG and pillar JSONs must be loaded first into Strapi

In [28]:
sdg_ids = get_ids("sdgs")
print(sdg_ids)

{'SDG 7 - Affordable and clean energy': 1, 'SDG 8 - Decent work and economic growth': 2, 'SDG 9 - Industry Innovation and Infrastructure': 3, 'SDG 11 - Sustainable Cities and Communities': 4, 'SDG 12 - Responsible production and consumption': 5, 'SDG 13 - Climate Action': 6, 'SDG 14 - Life below water': 7, 'SDG 15 - Life on land': 8, 'SDG 17 - Partnership for the goals': 9}


In [29]:
pillar_ids = get_ids("pillars")
print(pillar_ids)

{'1.5% New Green Jobs for Physical & Economic Resilience': 1, '30 x 30 Nature Based Solutions': 4, '90% Renewable Energy for All': 3, 'Climate Smart Map': 2}


In [30]:
country_ids = get_ids("countries")
print(sorted(country_ids.items(), key=lambda x: x[0]))

[('Anguilla', 919), ('Antigua and Barbuda', 920), ('Aruba', 918), ('Bahamas', 922), ('Barbados', 925), ('Belize', 923), ('Bermuda', 924), ('Bonaire, Sint Eustatius and Saba', 921), ('British Virgin Islands', 946), ('Cayman Islands', 928), ('Costa Rica', 926), ('Curaçao', 927), ('Dominica', 929), ('Dominican Republic', 930), ('Grenada', 932), ('Guadeloupe', 931), ('Guyana', 933), ('Haiti', 935), ('Honduras', 934), ('Jamaica', 936), ('Montserrat', 940), ('México', 939), ('Panama', 941), ('Puerto Rico', 948), ('Saint Kitts and Nevis', 937), ('Saint Lucia', 938), ('Saint Vincent and the Grenadines', 945), ('Suriname', 942), ('Trinidad and Tobago', 944), ('Turks and Caicos Islands', 943), ('Virgin Islands, U.S.', 947)]


## Clean Data and make JSON

In [1]:
# Countries that are not in the countries table in Strapi

print(
    set(df.select(pl.col("Operates in").str.split("; ").list.explode()).unique().to_numpy().ravel())
    - country_ids.keys()
)

NameError: name 'df' is not defined

In [35]:
countries_fix = {
    "Bonaire": "Bonaire, Sint Eustatius and Saba",
    "Cayman": "Cayman Islands",
    "Curacao": "Curaçao",
    "Japan": "",
    "Mexico": "México",
    "Regional": "",
    "St. Kitts & Nevis": "Saint Kitts and Nevis",
    "St. Vincent & the Grenadines": "Saint Vincent and the Grenadines",
    "Trinidad & Tobago": "Trinidad and Tobago",
    "Turks & Caicos Islands": "Turks and Caicos Islands",
    "US Virgin Islands": "Virgin Islands, U.S.",
}

df = df.with_columns(
    pl.col("Operates in")
    .str.split("; ")
    .list.eval(
        pl.when(pl.element().is_in(countries_fix.keys()))
        .then(pl.element().replace(countries_fix, default=None))
        .otherwise(pl.element())
    ),
)

In [39]:
# count the nulls
print(df.filter(pl.col("Operates in").list.eval(pl.element().is_null()).list.any()))

shape: (0, 12)
┌─────────┬─────────────┬─────────┬───────────┬───┬──────┬────────┬────────────────────┬───────────┐
│ Project ┆ Project     ┆ Project ┆ Objective ┆ … ┆ SDGs ┆ CCSA   ┆ Organization Type  ┆ More Info │
│ Name    ┆ Description ┆ Status  ┆ ---       ┆   ┆ ---  ┆ Pillar ┆ ---                ┆ ---       │
│ ---     ┆ ---         ┆ ---     ┆ str       ┆   ┆ str  ┆ ---    ┆ str                ┆ str       │
│ str     ┆ str         ┆ str     ┆           ┆   ┆      ┆ str    ┆                    ┆           │
╞═════════╪═════════════╪═════════╪═══════════╪═══╪══════╪════════╪════════════════════╪═══════════╡
└─────────┴─────────────┴─────────┴───────────┴───┴──────┴────────┴────────────────────┴───────────┘


In [60]:
# map pillar, SDG and countries to corresponding IDs
df_with_ids = df.with_columns(
    pl.col("SDGs").str.split("; ").list.eval(pl.element().replace(sdg_ids, default=None)),
    pl.col("CCSA Pillar").replace(pillar_ids, default=None),
    pl.col("Operates in").list.eval(pl.element().replace(country_ids, default=None)),
)

column_names = {
    "CCSA Pillar": "pillar",
    "Project Name": "name",
    "Project Description": "highlight",
    "Project Status": "status",
    "Project Needs": "funding",
    # "Account Name": "account",
    "Project Value": "amount",
    "SDGs": "sdgs",
    "Operates in": "countries",
}
# count the nulls
print(df_with_ids.filter(pl.col("Operates in").list.eval(pl.element().is_null()).list.any()))

shape: (7, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Project   ┆ Project   ┆ Project   ┆ Objective ┆ … ┆ SDGs      ┆ CCSA      ┆ Organizat ┆ More     │
│ Name      ┆ Descripti ┆ Status    ┆ ---       ┆   ┆ ---       ┆ Pillar    ┆ ion Type  ┆ Info     │
│ ---       ┆ on        ┆ ---       ┆ str       ┆   ┆ list[i64] ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ str       ┆           ┆   ┆           ┆ i64       ┆ str       ┆ str      │
│           ┆ str       ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Phase 1:  ┆ Caribbean ┆ Completed ┆ This      ┆ … ┆ [1, 2, …  ┆ 3         ┆ For-profi ┆ null     │
│ Solar     ┆ nations   ┆           ┆ project   ┆   ┆ 9]        ┆           ┆ t         ┆          │
│ Panel     ┆ need to   ┆           ┆ has been  ┆   ┆           ┆           

In [61]:
df.columns

['Project Name',
 'Project Description',
 'Project Status',
 'Objective',
 'Project Value',
 'Project Needs',
 'Operates in',
 'Source Country',
 'SDGs',
 'CCSA Pillar',
 'Organization Type',
 'More Info']

In [62]:
df_with_ids.select(*column_names.keys()).rename(column_names).write_json(
    "projects.json", pretty=True, row_oriented=True
)

In [63]:
len(df_with_ids)

121